# Testing KoDA fetching, parsing and transforming
This notebook explores how to fetch, parse and transform KoDA data for the prediction task.
The main source of how KoDa data is formatted and used is the [pykoda repo](https://github.com/KoDa-project/pykoda) from which large parts of the parsing code was adapted.

So far, the code is able to download static and realtime data and transform the realtime data into more compact data frames.

_TODO:_ For later convenience, there should probably also be a function which handles the entire process from (potentially) downloading the data to transforming it into a data frame.
That function should also handle deleting the downloaded zip files and the unzipped folders after the data has been transformed into feather files.


**Data exploration notes**:
- The static data is in GTFS txt files in a flat folder structure
- The realtime data is in Protocol Buffer files in a nested folder structure
- Some hours appear to have no realtime data (e.g. 2023-01-05-02) maybe some maintenance or downtime of the operator

**Outstanding questions**:
- Which features from `TripUpdates` are useful for our prediction task (also which aggregates to use)?
- Is there something useful in the static data for our prediction task or do we just use the realtime data?
- Are duplicates in the `TripUpdates` realtime data useful for our task?
    - It is not standardized how many updates are pushed per trip, but their information is likely useful
    - We'll try only using the latest tuple of (trip_id, stop_id, timestamp)
- Is there something in the `ServiceAlerts` or `VehiclePositions` realtime data that we need?

## Initial setup

In [1]:
import koda.koda_fetch as kf
import koda.koda_parse as kp
import koda.koda_transform as kt
from koda.koda_constants import OperatorsWithRT, FeedType

In [2]:
operator = OperatorsWithRT.X_TRAFIK
date = "2023-01-05"
feed_type = FeedType.TRIP_UPDATES
print(f"Fetching GTFS static data for {operator} on {date}")
static_zip_path = kf.fetch_gtfs_static_archive(operator, date)
print(f"Fetching GTFS realtime data for {operator} on {date}")
rt_zip_path = kf.fetch_gtfs_realtime_archive(operator, feed_type, date)

Fetching GTFS static data for OperatorsWithRT.X_TRAFIK on 2023-01-05
File already exists.
Fetching GTFS realtime data for OperatorsWithRT.X_TRAFIK on 2023-01-05
File already exists.


In [4]:
hour = "13"

static_folder_path = kp.unzip_gtfs_archive(static_zip_path)
print(f"Unzipped static data to {static_folder_path}")

rt_folder_path = kp.unzip_gtfs_archive(rt_zip_path)
print(f"Unzipped realtime data to {rt_folder_path}")

df = kt.read_rt_hour_to_df(operator, feed_type, date, hour)
print(f"Read {len(df)} rows from realtime data")
df

Unzipping ./dev_data/koda_download/xt_static_2023_01_05.7z
File already unzipped.
Unzipped static data to ./dev_data/koda_data\xt_static_2023_01_05
Unzipping ./dev_data/koda_download/xt_rt_2023_01_05.7z
File already unzipped.
Unzipped realtime data to ./dev_data/koda_data\xt_rt_2023_01_05
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T13.feather
Read 54402 rows from realtime data


,index,id,trip_id,start_date,schedule_relationship,timestamp,vehicle_id,stop_sequence,stop_id,arrival_delay,arrival_time,departure_delay,departure_time,arrival_uncertainty,departure_uncertainty
0,7,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,72,9022021483109001,456,1672919388,475,1672919407,0.0,0.0
1,17,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,82,9022021421038002,496,1672919959,507,1672919970,0.0,NaN
2,30,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,95,9022021421073002,394,1672920600,395,1672920601,NaN,NaN
3,31,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,96,9022021421042002,391,1672920626,391,1672920626,NaN,NaN
4,32,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,97,9022021421047002,388,1672920649,418,1672920679,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54397,3310,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,46,9022021480077001,117,1672926750,122,1672926755,NaN,NaN
54398,3311,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,47,9022021480064001,88,1672926782,90,1672926784,NaN,NaN
54399,3312,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,48,9022021480062001,64,1672926808,69,1672926813,NaN,NaN
54400,3313,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,49,9022021480029001,11,1672926865,33,1672926887,NaN,NaN


In [4]:
# Warning: This may take some time
df = kt.read_rt_day_to_df(operator, feed_type, date)
df

Reading xt TripUpdates 2023-01-05:   0%|          | 0/24 [00:00<?, ?it/s]

Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T0.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T1.feather
Reading 257 files with 14 processes


Reading xt TripUpdates 2023-01-05:  12%|█▎        | 3/24 [00:01<00:08,  2.36it/s]

Read 257 files
No data found in ./dev_data/koda_data/xt_rt_2023_01_05\xt\TripUpdates\2023\01\05\02
Reading 257 files with 14 processes


Reading xt TripUpdates 2023-01-05:  46%|████▌     | 11/24 [00:02<00:02,  5.79it/s]

Read 257 files
No data found in ./dev_data/koda_data/xt_rt_2023_01_05\xt\TripUpdates\2023\01\05\03
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T4.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T5.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T6.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T7.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T8.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T9.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T10.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T11.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T12.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T13.feather
Reading from ./dev_data/kod

Reading xt TripUpdates 2023-01-05: 100%|██████████| 24/24 [00:02<00:00,  8.53it/s]

Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T16.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T17.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T18.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T19.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T20.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T21.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T22.feather
Reading from ./dev_data/koda_data/xt_rt_2023_01_05/xt-tripupdates-2023-01-05T23.feather


,id,trip_id,start_date,schedule_relationship,timestamp,vehicle_id,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty
0,217990500811380121,217990000029636565,20230104,SCHEDULED,1672873041,9031021000444062,34,9022021484232001,21.0,1672872632,0.0,21,1672872632,0.0
1,217990500831603630,217990000029915771,20230104,SCHEDULED,1672873193,9031021001241753,32,9022021480209002,576.0,1672872652,0.0,576,1672872652,0.0
2,217990500831603630,217990000029915771,20230104,SCHEDULED,1672873193,9031021001241753,40,9022021480227002,516.0,1672873215,NaN,516,1672873215,NaN
3,217990500831603630,217990000029915771,20230104,SCHEDULED,1672873193,9031021001241753,41,9022021480219002,519.0,1672873247,NaN,520,1672873248,NaN
4,217990500831603630,217990000029915771,20230104,SCHEDULED,1672873193,9031021001241753,51,9022021480119007,454.0,1672874134,NaN,474,1672874154,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12044,217990500828590669,217990000029047775,20230106,SCHEDULED,1672959371,9031021001271556,43,9022021480359001,35.0,1672962602,NaN,39,1672962606,NaN
12045,217990500828590669,217990000029047775,20230106,SCHEDULED,1672959371,9031021001271556,44,9022021480467001,28.0,1672962651,NaN,34,1672962657,NaN
12046,217990500828590669,217990000029047775,20230106,SCHEDULED,1672959371,9031021001271556,45,9022021480427001,25.0,1672962682,NaN,28,1672962685,NaN
12047,217990500828590669,217990000029047775,20230106,SCHEDULED,1672959371,9031021001271556,46,9022021480242001,25.0,1672962739,NaN,32,1672962746,NaN


## Tests based on pre-processed feather files

In [1]:
import koda.koda_transform as kt
import pandas as pd

# Read the saved feather file
operator = "xt"
date = "2023-01-05"
feed_type = "TripUpdates"
hour = "13"
rt_feather_path = kt.get_rt_feather_path(operator, feed_type, date, hour)
df = pd.read_feather(rt_feather_path)
df

,index,id,trip_id,start_date,schedule_relationship,timestamp,vehicle_id,stop_sequence,stop_id,arrival_delay,arrival_time,departure_delay,departure_time,arrival_uncertainty,departure_uncertainty
0,7,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,72,9022021483109001,456,1672919388,475,1672919407,0.0,0.0
1,17,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,82,9022021421038002,496,1672919959,507,1672919970,0.0,NaN
2,30,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,95,9022021421073002,394,1672920600,395,1672920601,NaN,NaN
3,31,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,96,9022021421042002,391,1672920626,391,1672920626,NaN,NaN
4,32,217990500824825801,217990000029118861,20230105,SCHEDULED,1672919979,9031021000444433,97,9022021421047002,388,1672920649,418,1672920679,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54397,3310,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,46,9022021480077001,117,1672926750,122,1672926755,NaN,NaN
54398,3311,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,47,9022021480064001,88,1672926782,90,1672926784,NaN,NaN
54399,3312,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,48,9022021480062001,64,1672926808,69,1672926813,NaN,NaN
54400,3313,217990500806555531,217990000029012757,20230105,SCHEDULED,1672923554,9031021001271564,49,9022021480029001,11,1672926865,33,1672926887,NaN,NaN


In [2]:
print(f"Read {len(df)} rows from realtime data")

# Specify the columns to keep
columns_to_keep = [
    'trip_id', 'start_date', 'timestamp',
    'vehicle_id', 'stop_sequence', 'stop_id', 'arrival_delay',
    'arrival_time', 'departure_delay', 'departure_time'
]

# Extract the specified columns into a new DataFrame
df_filtered = df[columns_to_keep]

# Get average amount of updates per trip
average_updates = df_filtered.groupby('trip_id').size().mean()
# Get average amount of updates per trip and stop
average_updates_per_stop = df_filtered.groupby(['trip_id', 'stop_id']).size().mean()

average_updates_per_stop

Read 54402 rows from realtime data


np.float64(6.322873082287308)

In [3]:
deduplicated = kt.keep_only_latest_stop_updates(df_filtered)
deduplicated

,trip_id,start_date,timestamp,vehicle_id,stop_sequence,stop_id,arrival_delay,arrival_time,departure_delay,departure_time
0,217990000023580585,20230105,1672923568,9031021001271515,4,9022021480091001,21,1672923156,34,1672923169
1,217990000023580585,20230105,1672923568,9031021001271515,20,9022021480095001,-89,1672924071,0,1672924160
2,217990000023580585,20230105,1672923568,9031021001271515,26,9022021480112001,-145,1672924275,0,1672924420
3,217990000023580585,20230105,1672923568,9031021001271515,17,9022021480143001,13,1672923913,13,1672923913
4,217990000023580585,20230105,1672923568,9031021001271515,12,9022021480155001,109,1672923565,109,1672923565
...,...,...,...,...,...,...,...,...,...,...
8599,217990000029918992,20230105,1672923346,9031021000441726,14,9022021480460001,219,1672922861,219,1672922861
8600,217990000029918992,20230105,1672923466,9031021000441726,30,9022021480461001,215,1672923934,215,1672923934
8601,217990000029918992,20230105,1672923079,9031021000441726,10,9022021480635001,152,1672922543,152,1672922543
8602,217990000029918992,20230105,1672923466,9031021000441726,51,9022021480677001,-24,1672924880,0,1672924904


In [6]:
# Get average amount delays per trip
average_delays = deduplicated.groupby('trip_id')[['arrival_delay', 'departure_delay']].mean().reset_index()
average_delays

,trip_id,arrival_delay,departure_delay
0,217990000023580585,-18.185185,27.814815
1,217990000023581154,0.307692,6.269231
2,217990000023633371,-221.214286,-35.142857
3,217990000025164429,61.037037,77.333333
4,217990000025588572,50.709677,78.451613
...,...,...,...
285,217990000029906029,131.097561,144.682927
286,217990000029906077,122.219512,140.634146
287,217990000029916217,210.448276,218.413793
288,217990000029916276,164.423077,176.865385


In [4]:
deduped_df = kt.drop_tripupdates_duplicates(df_filtered)

average_delays = deduped_df.groupby('trip_id')[['arrival_delay', 'departure_delay']].mean().reset_index()


average_delays

,trip_id,arrival_delay,departure_delay
0,217990000023580585,-18.185185,27.814815
1,217990000023581154,0.307692,6.269231
2,217990000023633371,-221.214286,-35.142857
3,217990000025164429,61.037037,77.333333
4,217990000025588572,50.709677,78.451613
...,...,...,...
285,217990000029906029,131.097561,144.682927
286,217990000029906077,122.219512,140.634146
287,217990000029916217,210.448276,218.413793
288,217990000029916276,164.423077,176.865385


## Merging with static data
`pykoda` has a `get_data_range` function which shows how to merge the realtime GTFS TripUpdates with the static GTFS data.
Currently unsure if we need any static data for the prediction task.

In [7]:
import koda.koda_parse as kpa
import koda.koda_pipeline as kpi
from koda.koda_constants import StaticDataTypes, OperatorsWithRT, FeedType, route_types

operator = OperatorsWithRT.X_TRAFIK
date = "2023-01-05"
feed_type = FeedType.TRIP_UPDATES

tu_df = kpi.get_trip_updates_for_day(date, operator)
# Only keep trips that start on the specified date
tu_df = tu_df[tu_df['start_date'] == date.replace("-", "")]

trips_df = kpa.read_static_data_to_dataframe(operator, StaticDataTypes.TRIPS, date)
routes_df = kpa.read_static_data_to_dataframe(operator, StaticDataTypes.ROUTES, date)

# Cast trip_id to string
tu_df['trip_id'] = tu_df['trip_id'].astype(str)
trips_df['trip_id'] = trips_df['trip_id'].astype(str)
trips_df

get_trip_updates_for_day already completed for xt on 2023-01-05


Reading xt TripUpdates 2023-01-05: 100%|██████████| 24/24 [00:00<00:00, 78.31it/s] 


,route_id,service_id,trip_id,trip_headsign,direction_id,shape_id
0,9011021001900000,1,217990000029066217,NaN,1,1217990000023723009
1,9011021001900000,1,217990000029217873,NaN,1,1217990000023723058
2,9011021001900000,1,217990000029746103,NaN,1,1217990000029746020
3,9011021002000000,1,217990000028898037,NaN,1,1217990000028401901
4,9011021002000000,1,217990000028898062,NaN,1,1217990000028401901
...,...,...,...,...,...,...
4086,9011021301500000,47,217990000029825649,NaN,0,4217990000028139418
4087,9011021301500000,71,217990000029825669,NaN,0,4217990000028139392
4088,9011021301500000,72,217990000029825741,NaN,0,4217990000028139392
4089,9011021301500000,66,217990000029825804,NaN,0,4217990000028139392


In [8]:
# Join with static data on trip_id
tu_df = tu_df.merge(trips_df, on='trip_id', how='inner')
tu_df

,id,trip_id,start_date,schedule_relationship,timestamp,vehicle_id,stop_sequence,stop_id,arrival_delay,arrival_time,arrival_uncertainty,departure_delay,departure_time,departure_uncertainty,route_id,service_id,trip_headsign,direction_id,shape_id
0,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873512,9031021000444432,2,9022021421028002,-17.0,1672873550,NaN,0,1672873567,NaN,9011021010000000,23,NaN,1,1217990000025036410
1,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873512,9031021000444432,49,9022021483111002,231.0,1672875620,NaN,231,1672875620,NaN,9011021010000000,23,NaN,1,1217990000025036410
2,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873512,9031021000444432,50,9022021483110002,213.0,1672875653,NaN,213,1672875653,NaN,9011021010000000,23,NaN,1,1217990000025036410
3,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873553,9031021000444432,3,9022021421035002,-7.0,1672873610,NaN,0,1672873617,NaN,9011021010000000,23,NaN,1,1217990000025036410
4,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873607,9031021000444432,3,9022021421035002,-10.0,1672873607,0.0,0,1672873617,NaN,9011021010000000,23,NaN,1,1217990000025036410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835533,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,37,9022021480087001,41.0,1672962021,NaN,46,1672962026,NaN,9011021090300000,11,NaN,0,1217990000029594424
835534,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,38,9022021480195001,17.0,1672962069,NaN,23,1672962075,NaN,9011021090300000,11,NaN,0,1217990000029594424
835535,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,39,9022021480113001,-13.0,1672962088,NaN,8,1672962109,NaN,9011021090300000,11,NaN,0,1217990000029594424
835536,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,40,9022021480197001,-6.0,1672962128,NaN,0,1672962134,NaN,9011021090300000,11,NaN,0,1217990000029594424


In [9]:
# Join with static data on route_id
tu_df = tu_df.merge(routes_df, on='route_id', how='inner')
# Map route_type to strings with route_types dict
tu_df['vehicle_type'] = tu_df['route_type'].map(route_types)
tu_df

,id,trip_id,start_date,schedule_relationship,timestamp,vehicle_id,stop_sequence,stop_id,arrival_delay,arrival_time,...,service_id,trip_headsign,direction_id,shape_id,agency_id,route_short_name,route_long_name,route_type,route_desc,vehicle_type
0,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873512,9031021000444432,2,9022021421028002,-17.0,1672873550,...,23,NaN,1,1217990000025036410,217990000000001393,100,NaN,700,NaN,Bus Service
1,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873512,9031021000444432,49,9022021483111002,231.0,1672875620,...,23,NaN,1,1217990000025036410,217990000000001393,100,NaN,700,NaN,Bus Service
2,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873512,9031021000444432,50,9022021483110002,213.0,1672875653,...,23,NaN,1,1217990000025036410,217990000000001393,100,NaN,700,NaN,Bus Service
3,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873553,9031021000444432,3,9022021421035002,-7.0,1672873610,...,23,NaN,1,1217990000025036410,217990000000001393,100,NaN,700,NaN,Bus Service
4,217990500824411634,217990000029109589,20230105,SCHEDULED,1672873607,9031021000444432,3,9022021421035002,-10.0,1672873607,...,23,NaN,1,1217990000025036410,217990000000001393,100,NaN,700,NaN,Bus Service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835533,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,37,9022021480087001,41.0,1672962021,...,11,NaN,0,1217990000029594424,217990000000001393,3,NaN,700,Gävle,Bus Service
835534,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,38,9022021480195001,17.0,1672962069,...,11,NaN,0,1217990000029594424,217990000000001393,3,NaN,700,Gävle,Bus Service
835535,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,39,9022021480113001,-13.0,1672962088,...,11,NaN,0,1217990000029594424,217990000000001393,3,NaN,700,Gävle,Bus Service
835536,217990500830763102,217990000029901675,20230105,SCHEDULED,1672959566,9031021001271545,40,9022021480197001,-6.0,1672962128,...,11,NaN,0,1217990000029594424,217990000000001393,3,NaN,700,Gävle,Bus Service
